# Setup Geral

Se estiver executando este exercício no Google Colab, execute as próximas duas células. 

Caso esteja executando localmente, não é necessário executar mas certifique-se de que o **pyspark** está instalado e configurado em sua máquina.

In [1]:
%%bash

# Instal Java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelea

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Teste

O teste a ser realizado é composto de 3 partes:

- um exercício de programação em Python
- alguns exercícios de programação em SQL
- alguns exercícios de programação com PySpark

Você pode escolher qual das partes do exercício vai fazer primeiro. Todo o exercício deve ser completado no período de 90 minutos.

# Python

In [3]:
# SETUP
nomes_alunos = [
    ('Maria', 1),
    ('João', 2),
    ('Pedro', 3),
    ('Gabriella', 4),
    ('Giovana', 5),
    ('Arthur', 6)
]

notas_alunos = {
    1: 9.5,
    2: 5.1,
    3: 8.7,
    4: 7.1,
    5: 4.8,
    6: 6.3
}

Implemente uma função que recebe uma lista de nomes de alunos, um dicionário de notas dos mesmo, sendo que essas estruturas se relacionam por um ID.

A função deve retornar em ordem alfabética, o nome dos alunos que obtiveram notas maior ou igual de uma nota de corte informada.

**[DICA]**: Para realizar a atividade, pense em iterar por cada elemento tanto do dicionário quanto da lista de tuplas e fazer as comparações dentro das iterações.

In [13]:
def filtra_alunos_acima_corte(alunos, notas, nota_corte):
    output = []
    for id, nota in notas_alunos.items():
      if nota >= nota_corte:
        #print(f'Nota {nota} do aluno de id {id} é maior que a de corte {nota_corte}')
        aluno = [x[0] for x in nomes_alunos if x[1] == id][0]
        output.append(aluno)
    output.sort()
    print(f'A lista de alunos, em ordem alfabética, que tiveram nota maior ou igual à nota de corte {nota_corte} é: ')
    return output

    
filtra_alunos_acima_corte(nomes_alunos, notas_alunos, 6)

A lista de alunos, em ordem alfabética, que tiveram nota maior ou igual à nota de corte 6 é: 


['Arthur', 'Gabriella', 'Maria', 'Pedro']

# SQL

**Setup**


In [14]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1039  100  1039    0     0   6534      0 --:--:-- --:--:-- --:--:--  6575
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533k  100  533k    0     0  2144k      0 --:--:-- --:--:-- --:--:-- 2144k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0   898k      0 --:--:-- --:--:-- --:--:--  895k


In [15]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [16]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

**Função para execução de queries**

In [17]:
def q(query, n=30):
    return spark.sql(query).show(n=n, truncate=False)

Para executar alguma consulta, basta colocar seu código sql dentro da função q como no exemplo abaixo:

```python
q("""
    SELECT *
    FROM usuarios
""")
```

e o resultado será exibido na tela.

---

Nesta parte da atividade, você vai trabalhar com três tabelas:

- produtos
- usuarios
- vendas

Use-as para responder às perguntas a seguir.

1) Qual foi a quantidade de vendas nos estados de Minas Gerais e São Paulo para cada ano e mês?

In [52]:
q("""
    SELECT year(v.data_compra) ano, month(v.data_compra) mes, u.estado, COUNT(v.quantidade) total
    FROM vendas v
    INNER JOIN usuarios u
      ON v.cod_usuario = u.cod_usuario
    WHERE u.estado IN ('Minas Gerais', 'São Paulo')
    GROUP BY ano, mes, u.estado
    ORDER BY ano DESC, mes DESC
""")

+----+---+------------+-----+
|ano |mes|estado      |total|
+----+---+------------+-----+
|2021|7  |São Paulo   |11   |
|2021|7  |Minas Gerais|5    |
|2021|6  |São Paulo   |6    |
|2021|6  |Minas Gerais|3    |
|2021|5  |São Paulo   |2    |
|2021|5  |Minas Gerais|4    |
|2021|4  |Minas Gerais|6    |
|2021|4  |São Paulo   |3    |
|2021|3  |São Paulo   |1    |
|2021|3  |Minas Gerais|1    |
|2021|2  |Minas Gerais|6    |
|2021|2  |São Paulo   |1    |
|2021|1  |Minas Gerais|2    |
|2020|12 |Minas Gerais|1    |
|2020|11 |Minas Gerais|3    |
|2020|10 |São Paulo   |4    |
|2020|10 |Minas Gerais|1    |
|2020|9  |Minas Gerais|3    |
|2020|9  |São Paulo   |2    |
|2020|7  |Minas Gerais|2    |
|2020|6  |Minas Gerais|3    |
|2020|6  |São Paulo   |1    |
|2020|5  |Minas Gerais|3    |
|2020|4  |Minas Gerais|1    |
|2020|4  |São Paulo   |1    |
|2020|3  |Minas Gerais|1    |
|2020|3  |São Paulo   |1    |
|2020|2  |Minas Gerais|1    |
|2020|1  |São Paulo   |1    |
|2020|1  |Minas Gerais|1    |
+----+---+

2) Quais são os usuários por Estado que mais compraram em todo o período analisado e qual foi o número de compras realizadas, a quantidade total de itens comprados e valor total pago por usuário?

**[DICA]**: perceba que a primeira parte da pergunta exige uma contagem do grão *usuário* e a segunda exige uma contagem do grão *vendas*.

In [79]:
q("""
    SELECT usuario, estado, MAX(compras) compras, total_quantidades, ROUND(total_valor, 2) FROM (
    SELECT u.cod_usuario usuario, u.estado, COUNT(v.quantidade) compras, SUM(v.quantidade) total_quantidades, SUM(v.valor) total_valor
    FROM vendas v
    INNER JOIN usuarios u
      ON v.cod_usuario = u.cod_usuario
    GROUP BY usuario, u.estado
    )
    GROUP BY usuario, estado, total_quantidades, total_valor
    ORDER BY compras DESC
""")

+-------+-------------------+-------+-----------------+---------------------+
|usuario|estado             |compras|total_quantidades|round(total_valor, 2)|
+-------+-------------------+-------+-----------------+---------------------+
|2811   |Maranhão           |13     |117              |184101.89            |
|3403   |Rio Grande do Norte|10     |94               |136347.01            |
|2843   |Paraná             |10     |114              |137228.01            |
|2340   |Mato Grosso        |9      |112              |165197.3             |
|1074   |Santa Catarina     |9      |66               |159165.18            |
|2981   |Bahia              |9      |70               |126282.4             |
|28     |Rio Grande do Sul  |9      |76               |29114.45             |
|3090   |Minas Gerais       |9      |93               |95697.3              |
|700    |Goiás              |9      |74               |142389.09            |
|2551   |Santa Catarina     |9      |88               |187513.66

3) Quais são os usuários que não fizeram nenhuma compra?

In [87]:
q("""
    SELECT DISTINCT u.cod_usuario
    FROM usuarios u
    LEFT JOIN vendas v
      ON v.cod_usuario = u.cod_usuario
    WHERE v.cod_usuario IS NULL;
""")

+-----------+
|cod_usuario|
+-----------+
|19984      |
|25517      |
|41409      |
|68090      |
|87120      |
|195413     |
|200379     |
|219523     |
|246097     |
|362827     |
|371765     |
|417190     |
|485130     |
|91785      |
|67782      |
|90817      |
|18654      |
|76143      |
|54264      |
|69048      |
|63155      |
|18800      |
|22521      |
|5614       |
|22188      |
|50732      |
|149257     |
|206380     |
|221801     |
|224325     |
+-----------+
only showing top 30 rows



4) Qual é o ticket médio (média de valor gasto) e o número total de usuários que fizeram pelo menos uma compra *por faixa etária*?

In [93]:
q("""
    SELECT u.faixa_etaria, ROUND(AVG(v.valor), 2) ticket_medio, COUNT(u.cod_usuario) total_usuarios
    FROM usuarios u
    LEFT JOIN vendas v
      ON v.cod_usuario = u.cod_usuario
    WHERE v.cod_usuario IS NOT NULL
    GROUP BY u.faixa_etaria
""")

+------------------+------------+--------------+
|faixa_etaria      |ticket_medio|total_usuarios|
+------------------+------------+--------------+
|Entre 22 a 27 anos|12203.02    |64            |
|Entre 10 a 15 anos|8890.13     |43            |
|Entre 16 a 21 anos|12725.95    |50            |
|Entre 50 a 61 anos|10849.48    |80            |
|Entre 37 a 49 anos|16727.19    |42            |
|Entre 62 a 70 anos|10012.63    |39            |
|Entre 28 a 36 anos|8233.04     |56            |
|Mais de 70 anos   |11303.1     |29            |
+------------------+------------+--------------+



In [94]:
spark.stop()

# PySpark

**setup**:

In [95]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

mkdir: cannot create directory ‘bases_teste’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1039  100  1039    0     0   6790      0 --:--:-- --:--:-- --:--:--  6790
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533k  100  533k    0     0  1516k      0 --:--:-- --:--:-- --:--:-- 1516k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0   964k      0 --:--:-- --:--:-- --:--:--  964k


In [110]:
# Setup Spark Session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [111]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

Responda às perguntas a seguir utilizando **Spark DATAFRAMES**.

1) Qual foi o total de compras realizadas, o total de itens comprados e a receita total obtida em todo o período analisado?

In [117]:
vendas.agg(count("cod_usuario").alias("total_compras"), sum("quantidade").alias("total_itens"), format_number(sum("valor"), 2).alias("total_receita")).show()

+-------------+-----------+--------------+
|total_compras|total_itens| total_receita|
+-------------+-----------+--------------+
|        20000|     209149|215,849,001.66|
+-------------+-----------+--------------+



2) Quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga?

In [120]:
produtos.show(5)
vendas.show(5)
usuarios.show(5)

+-----------+--------------------+--------------------+-------------+
|cod_produto|        nome_produto|   categoria_produto|valor_produto|
+-----------+--------------------+--------------------+-------------+
|          1|Notebook Asus Int...|          Tecnologia|       4590.9|
|          2|Bicicleta Caloi A...|               Lazer|      1345.78|
|          3|  Smartphone Samsung|          Tecnologia|       2049.5|
|          4|      Mochila Targus|Material de escri...|         58.9|
|          5|   Escrivaninha em L|Material de escri...|        437.0|
+-----------+--------------------+--------------------+-------------+
only showing top 5 rows

+-----------+-----------+-----------+----------+-------+
|cod_usuario|cod_produto|data_compra|quantidade|  valor|
+-----------+-----------+-----------+----------+-------+
|        927|         10| 2020-10-19|         4| 2799.6|
|       1544|         10| 2021-03-28|         2| 1399.8|
|       2833|          2| 2020-09-13|        20|26915.6|
|  

In [160]:
full_data = produtos.join(vendas, produtos.cod_produto == vendas.cod_produto, 'inner').drop(vendas.cod_produto).join(usuarios, usuarios.cod_usuario == vendas.cod_usuario, 'inner').drop(vendas.cod_usuario)
full_data.select('estado').distinct().show()

+-------------------+
|             estado|
+-------------------+
|     Santa Catarina|
| Mato Grosso do Sul|
|              Goiás|
|        Mato Grosso|
|              Ceará|
|              Piauí|
|             Paraná|
|              Bahia|
|         Pernambuco|
|  Rio Grande do Sul|
|Rio Grande do Norte|
|          São Paulo|
|       Minas Gerais|
|           Maranhão|
|          Tocantins|
|            Paraíba|
|              Amapá|
+-------------------+



In [161]:
full_data.show(5)

+-----------+--------------------+-------------------+-------------+-----------+----------+-------+-----------+-------------+------------------+--------------------+-----------------+
|cod_produto|        nome_produto|  categoria_produto|valor_produto|data_compra|quantidade|  valor|cod_usuario|data_cadastro|      faixa_etaria|              cidade|           estado|
+-----------+--------------------+-------------------+-------------+-----------+----------+-------+-----------+-------------+------------------+--------------------+-----------------+
|         18|Lenço umedecido T...|Produtos de limpeza|          7.9| 2021-06-09|        11|   86.9|       2441|   2019-04-16|Entre 16 a 21 anos|   Cajueiro da Praia|            Piauí|
|         17|Monitor LG 19 pol...|         Tecnologia|        584.1| 2019-10-28|         7| 4088.7|       2183|   2019-03-08|Entre 28 a 36 anos|São Francisco do Sul|   Santa Catarina|
|          8| Cafeteira Nespresso|   Eletrodomesticos|       219.57| 2021-03-17|

In [168]:
full_data.filter(full_data.estado.isin('São Paulo', 'Minas Gerais', 'Santa Catarina', 'Rio Grande do Sul', 'Paraná')).groupBy('nome_produto').agg(count('*').alias("mais_comprados"), sum("quantidade").alias("total_itens"), format_number(sum("valor"), 2).alias("valor_total"), format_number(avg("valor"), 2).alias("media_valor") ).sort(desc("mais_comprados")).show(3)

+--------------------+--------------+-----------+-----------+-----------+
|        nome_produto|mais_comprados|total_itens|valor_total|media_valor|
+--------------------+--------------+-----------+-----------+-----------+
|Alcool em Gel 70%...|            26|        280|   1,397.20|      53.74|
|Notebook Asus Int...|            14|        134| 615,180.60|  43,941.47|
|Monitor LG 19 pol...|            14|         96|  56,073.60|   4,005.26|
+--------------------+--------------+-----------+-----------+-----------+
only showing top 3 rows



3) Para cada produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago por eles?

In [163]:
full_data.groupBy('nome_produto').agg(count('cod_usuario').alias("quantidade_usuarios"), min('valor').alias("menor_valor"), max('valor').alias("maior_valor")).show(truncate=False)

+-----------------------------------+-------------------+-----------+-----------+
|nome_produto                       |quantidade_usuarios|menor_valor|maior_valor|
+-----------------------------------+-------------------+-----------+-----------+
|Bicicleta Caloi Aro 29             |15                 |2691.56    |26915.6    |
|Cafeteira Nespresso                |21                 |219.57     |3732.69    |
|Notebook Asus Intel Core i7        |29                 |4590.9     |82636.2    |
|Aparelho de Barbear OneBlade Philco|13                 |129.9      |2598.0     |
|SPA Intel 700 L                    |27                 |5999.1     |119982.0   |
|Jogo Mortal Kombat 11 PS4          |18                 |49.9       |848.3      |
|Umidificador de Ar Cadence         |21                 |77.5       |1550.0     |
|Blusa Lacoste                      |18                 |138.99     |2779.8     |
|Alcool em Gel 70% Johnson          |36                 |4.99       |99.8       |
|Monitor LG 19 p

4) Construa uma tabela unificada com todas as colunas dos 3 datasets (sem colunas repetidas) e escreva essa tabela localmente em formato **parquet** particionando pelo estado.

In [164]:
full_data.write.partitionBy('estado').parquet('full_data')

# FIM!

In [165]:
!pip install nbconvert

In [166]:
%%shell
jupyter nbconvert --to html /content/teste_engenharia_de_dados_a3data_ebraim.ipynb

[NbConvertApp] Converting notebook /content/teste_engenharia_de_dados_a3data_ebraim.ipynb to html
[NbConvertApp] Writing 333657 bytes to /content/teste_engenharia_de_dados_a3data_ebraim.html
